## Original post
https://python.langchain.com/v0.1/docs/use_cases/sql/agents/


In [3]:
from langchain_community.utilities import SQLDatabase

In [4]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

# Agent

In [24]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_aws import BedrockLLM, ChatBedrock

In [27]:
model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
#model_id = "meta.llama3-2-3b-instruct-v1:0"
#llm = BedrockLLM(model_id = model_id, model_kwargs={"temperature": 0},)
llm =ChatBedrock(model_id = model_id, model_kwargs={"temperature": 0})

agent_executor = create_sql_agent(llm, db=db, verbose=True)

In [30]:
response = agent_executor.invoke("Dame el total de ventas por país. Qué pais vende mas?"
    #"List the total sales per country. Which country's customers spent the most?"
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: 
Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackBased on the list of tables, it seems that the "Invoice" table might contain the information we need about sales by country. Let's check its schema to confirm.

Action: sql_db_schema
Action Input: Invoice

CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("InvoiceId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows from Invoice table:
InvoiceId	CustomerId	InvoiceDate	BillingAddress	BillingCity	BillingState	BillingCountry	BillingPostalCode	Total
1	2	2021-01-01 00:00:00	Theodor-Heuss-Str

In [35]:
print("Pregunta: ",response.get("input"))
print("Respuesta: ")
print(response.get("output"))

Pregunta:  Dame el total de ventas por país. Qué pais vende mas?
Respuesta: 
El total de ventas por país, ordenado de mayor a menor, es el siguiente:

1. USA: $523.06
2. Canada: $303.96
3. France: $195.10
4. Brazil: $190.10
5. Germany: $156.48
6. United Kingdom: $112.86
7. Czech Republic: $90.24
8. Portugal: $77.24
9. India: $75.26
10. Chile: $46.62

El país que vende más es Estados Unidos (USA) con un total de ventas de $523.06.


In [37]:
response = agent_executor.invoke("que artista vendio más? que disco es el mas comprado?")
print("Pregunta: ",response.get("input"))
print("Respuesta: ")
print(response.get("output"))



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: 
Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Based on the tables available, we'll need to focus on the Artist, Album, and InvoiceLine tables to answer this question. Let's first check the schema of these tables.

Action: sql_db_schema
Action Input: Artist, Album, InvoiceLine

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "InvoiceLine" (
	"Inv

In [38]:
response = agent_executor.invoke("Describe las tablas a las que tienes acceso")
print("Pregunta: ",response.get("input"))
print("Respuesta: ")
print(response.get("output"))



> Entering new SQL Agent Executor chain...
To answer this question, I need to first list all the tables in the database and then get the schema for each table. Let's start by listing the tables.

Action: sql_db_list_tables
Action Input: 
Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackNow that I have a list of all the tables in the database, I can describe them by getting the schema for each table. I'll use the sql_db_schema tool to get this information for all tables.

Action: sql_db_schema
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	